In [16]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import datetime
import os
import sys

In [17]:
# MinIO Storage Settings
MINIO_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
MINIO_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
LOGS_BUCKET = "logs"
REPORTS_BUCKET = "report"
LOG_FILE = "service_error_down.txt" # This variable will be used throughout the script

# LLM Info
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

In [18]:
# Read log txt file
with open(LOG_FILE, "r") as f:
    logs = f.read().strip()
print(logs)

HTTP SERVER HEALTH REPORT\n=========================\n[Basic Information]\n- Hostname: aiops\n- OS: RedHat 9.5\n\n[Web Server Status]\n- Installed: Yes\n- Service Status: UNKNOWN\n- Listening on Port 80: No\n\n[Security Configuration]\n\n\n[Connectivity]\n- HTTP Response: -1\n- Config Valid: Valid\n


In [19]:

def query_model_for_report(logs):
    """Queries the AI model to generate a full incident report."""
    print(f"\n--> Step 1: Querying AI model for full report...")
    prompt = f"""
Generate a concise, structured AIOps incident report based on the following logs. The service name must be the short, executable name (e.g., 'httpd').
The report must be in Markdown format and include:
1. Key metadata: INCIDENT ID, DETECTED, SEVERITY, STATUS, AFFECTED SERVICE, AFFECTED HOST, SUMMARY, and KEY ERROR LOG.
2. A ROOT CAUSE section explaining the 'why'.
3. A REMEDIATION PLAYBOOK section with exact, numbered shell commands to fix the issue.
4. A VALIDATION section with a command to confirm the fix.

--- LOGS START ---
{logs}
--- LOGS END ---
"""
    headers = {"Content-Type": "application/json"}
    payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": 512}
    
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status()
        result = response.json()
        if 'choices' in result and result['choices']:
            report_text = result['choices'][0].get('text', 'Error: Could not extract report.')
            print("    ✅ AI analysis complete.")
            return report_text
    except Exception as e:
        print(f"    ❌ Error querying model: {e}")
    return None

def save_report_to_json(report_text, filename="report.json"):
    """Saves the raw report text into a JSON file."""
    print(f"\n--> Step 2: Saving full report to '{filename}'...")
    try:
        report_data = {"report_content": report_text}
        with open(filename, 'w') as f:
            json.dump(report_data, f, indent=4)
        print(f"    ✅ Successfully saved full report.")
        return filename
    except Exception as e:
        print(f"    ❌ Error saving report to JSON: {e}")
    return None

def save_report_to_file(report_text, filename="incident_report.txt"):
    """Saves the incident report to a text file."""
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(report_text)
        print(f"    💾 Report saved to '{filename}'.")
    except Exception as e:
        print(f"    ❌ Error saving report to file: {e}")


In [20]:
def upload_to_minio(s3_client, bucket, object_name, content):
    """Uploads content to a specified MinIO bucket."""
    print(f"--> Uploading '{object_name}' to bucket '{bucket}'...")
    try:
        s3_client.put_object(Body=content.encode('utf-8'), Bucket=bucket, Key=object_name)
        print(f"    ✅ Successfully uploaded '{object_name}'.")
        return True
    except Exception as e:
        print(f"    ❌ Error uploading to MinIO: {e}")
        return False

In [21]:
def main():
    
    s3_client = boto3.client(
    's3', 
    endpoint_url=MINIO_ENDPOINT, 
    aws_access_key_id=MINIO_ACCESS_KEY, 
    aws_secret_access_key=MINIO_SECRET_KEY
    )
    
    incident_report = query_model_for_report(logs)
    if not incident_report:
        sys.exit("Pipeline stopped: Could not generate AI report.")
        
    save_report_to_file(incident_report)

    with open('incident_report.txt', 'r', encoding='utf-8') as file:
        report_file = file.read()

    if not upload_to_minio(s3_client, REPORTS_BUCKET, f"incident_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt", report_file):
        sys.exit("Pipeline stopped: Failed to upload report to MinIO.")

if __name__ == "__main__":
    print("==============================================")
    print("AIOps PIPELINE STAGE: Generate Incident Report")
    print("==============================================")
    main()
    print("\n==============================================")
    print("                STAGE COMPLETE")
    print("==============================================")

AIOps PIPELINE STAGE: Generate Incident Report

--> Step 1: Querying AI model for full report...


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


    ✅ AI analysis complete.
    💾 Report saved to 'incident_report.txt'.
--> Uploading 'incident_report_20250617_093758' to bucket 'report'...
    ✅ Successfully uploaded 'incident_report_20250617_093758'.

                STAGE COMPLETE
